In [33]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import time
import itertools
from itertools import product
import torch

print('mps: ', torch.backends.mps.is_available())

options = {
 "WLSACCESSID":"a4353fb7-f95b-4075-b288-ca3f60983b36",
"WLSSECRET":"d894d460-2dac-4210-8c40-c91c68ecfb13",
"LICENSEID":2562382
}

# Check if MPS is available
if torch.backends.mps.is_available():
    device = torch.device("mps")  # Use the Metal backend
else:
    device = torch.device("cpu")  # Fallback to CPU

mps:  True


In [34]:
class BundledChoiceKP:
    def __init__(self,  num_agents , num_objects , lambda_star = None, num_simulations = 20, 
                 random_seed = 4,  sigma = 1, max_capacity = 100, generate_data = True):

        self.num_agents = num_agents
        self.num_objects = num_objects
        self.num_simulations = num_simulations


        self.random_seed = random_seed
        self.device = device
        self.agents_si = torch.kron(torch.ones(self.num_simulations, dtype = torch.int), torch.arange(self.num_agents))

        
        if lambda_star is not None:           
            # True parameters
            self.K_MOD = len(lambda_star)
            self.lambda_star_np = lambda_star
            self.lambda_star = torch.tensor(lambda_star, device=device, dtype=torch.float)

        else:                                 
            self.K_MOD = None
            self.lambda_star_np = None
            self.lambda_star = None

        if generate_data is True:
            # Set manual seed
            torch.manual_seed(self.random_seed)

            ### Modular characteristics
            self.φ_i_j_k = torch.normal(0, 1, size=(self.num_agents, self.num_objects, self.K_MOD), 
                                        device=device, dtype=torch.float)

            ### Knapsack Constraints
            self.weight_j = torch.randint(1, 100, (self.num_objects,), device=device, dtype = torch.int)
            self.capacity_i = torch.randint(1, max_capacity, (self.num_agents,), device=device, dtype=torch.int) 
        else:
            self.φ_i_j_k = None
            self.weight_j = None
            self.capacity_i = None
        
    
        ### Estimation
        self.φ_hat_k = None
        self.φ_hat_i_k = None
        self.value_LP = None

        self.eps_i_j = sigma * torch.normal(0, 1, size=(self.num_agents, self.num_objects), device=device, dtype=torch.float)
        self.eps_si_j = sigma * torch.normal(0, 1, size=(self.num_simulations * self.num_agents, self.num_objects), 
                                             device=device, dtype=torch.float)

    def load_data(self, characteristic_i_j, weight_j, capacity_i, matching_i_j):
        # set the characteristic function (torch tensor)
        self.φ_i_j_k  = torch.tensor(characteristic_i_j, dtype=torch.float32, device=self.device)

        # set the weight of the objects (torch tensor int64)
        self.weight_j = torch.tensor(weight_j, dtype=torch.int64, device=self.device)
        self.capacity_i = torch.tensor(capacity_i, dtype=torch.int64, device=self.device)
        matching_i_j = torch.tensor(matching_i_j, dtype=torch.bool, device=self.device)
        self.φ_hat_k = (self.φ_i_j_k * matching_i_j.unsqueeze(2)).sum((0,1))

    

In [35]:
def linear_knapsack(self, idx, lambda_k , max_weight = None, p_j = None, eps_i_j = None, return_value = False):
    
    ### Compute the values
    profit_i_j = self.φ_i_j_k[idx]  @  lambda_k 

    if p_j is not None:
        profit_i_j -= p_j.unsqueeze(0)
    if eps_i_j is not None:
        profit_i_j += eps_i_j

    if max_weight is None:
        max_weight = int(self.capacity_i[idx].max())
    
    ### Fill in the DP table
    value_i_j_w = torch.zeros((len(idx), self.num_objects +1, max_weight +1) ,device=device, dtype=torch.float)
    weight_states = torch.arange(max_weight + 1, device = device)

    # print type of weight_states
    for j in range(self.num_objects):
        value_i_j_w[:, j+1, :] = torch.where(self.weight_j[j] <= weight_states, 
                                    torch.maximum(profit_i_j[:, j].unsqueeze(1) + value_i_j_w[:, j, weight_states - self.weight_j[j]],
                                                value_i_j_w[:, j, :]), 
                                        value_i_j_w[:, j, :])

    ### Backtrack to find the items
    residual_weight = self.capacity_i[idx]
    B_i_j_star = torch.zeros((len(idx), self.num_objects), device=device, dtype= bool)

    for j in range(self.num_objects,0,-1):

        pick_j = (value_i_j_w[torch.arange(len(idx)), j, residual_weight] > 
                    value_i_j_w[torch.arange(len(idx)), j-1, residual_weight])
        
        B_i_j_star[:, j-1] = pick_j
        residual_weight -= pick_j * self.weight_j[j-1]

    if return_value:
        return B_i_j_star, value_i_j_w[torch.arange(len(idx)), -1 , self.capacity_i[idx]]
    else:
        return B_i_j_star 

BundledChoiceKP.linear_knapsack = linear_knapsack

## Discrete choice

In [36]:
def generate_data_bundle_choice(self, add_noise = False):

    noise = self.eps_i_j if add_noise else None

    B_i_j_star = self.linear_knapsack(torch.arange(self.num_agents), self.lambda_star, eps_i_j = noise)

    self.φ_hat_i_k = (self.φ_i_j_k * B_i_j_star.unsqueeze(2)).sum(1)
    self.φ_hat_k = self.φ_hat_i_k.sum(0)

    print("# characteristics: ",self.K_MOD)
    print("φ_hat_k: ", self.φ_hat_k.cpu().numpy())

BundledChoiceKP.generate_data_bundle_choice = generate_data_bundle_choice

In [37]:
def estimate_minmaxregret_BChoice(self, max_iters = 100 ,tol = 1e-2 ):
      with gp.Env(params=options) as env:
          with gp.Model(env=env) as model:
              ### Initialize
              # Create variables
              lambda_k = model.addVars(self.K_MOD - 1, lb= 0, ub = 1e8 , name="parameters")
              u_i = model.addVars(self.num_agents, lb = 0, ub = 1e8 , name="utilities")

              # Set objective and initial constraints
              model.setObjective(gp.quicksum( self.φ_hat_k[k+1] * lambda_k[k] for k in range(self.K_MOD - 1))
                                  - u_i.sum(), GRB.MAXIMIZE)

              model.addConstrs(
                            (u_i[i] >= gp.quicksum((self.φ_hat_i_k[i, k+1]) * lambda_k[k]  for k in range(self.K_MOD - 1))
                            + self.φ_hat_i_k[i, 0]
                            for i in range(self.num_agents))
                              )
              
              # Solve master problem 
              model.setParam('OutputFlag', 0)
              model.optimize()
              theta_solution = np.array(model.x)

              iter = 0
              while iter < max_iters:
                  print('################')
                  print(f"ITER: {iter}, parameters: {theta_solution[:self.K_MOD-1].round(3)}")

                  ### Pricing problem
                  lambda_k_iter = torch.cat((torch.tensor([1.0], device= device),
                                               torch.tensor(theta_solution[:self.K_MOD-1], device= device , dtype=torch.float32)))


                  B_star_i = self.linear_knapsack(torch.arange(self.num_agents), lambda_k_iter)

                  val_i =  (self.φ_i_j_k * B_star_i.unsqueeze(2)).sum(1) @ lambda_k_iter

                  ### Stop if certificate holds
                  certificate_i = val_i.cpu().numpy() - theta_solution[self.K_MOD-1:]
                  max_certificate = np.max(certificate_i)

                  print("Value of LP:   ", model.objVal)
                  print(f"Reduced cost: {max_certificate}")

                  if max_certificate < tol:
                    primal_solution = np.array(model.x)
                    dual_solution = np.array(model.pi)
                    print('#############################################')
                    print('#############################################')
                    print("Solution:       ", primal_solution[:self.K_MOD-1].round(3))
                    print("True parameters:", self.lambda_star_np[1:])
                    print('###############')
                    return primal_solution

                    

                  ### Master problem
                  # Add constraints

                  φ_i_k_star = (self.φ_i_j_k * B_star_i.unsqueeze(2)).sum(1)

                  model.addConstrs(
                            (u_i[i] >= gp.quicksum((φ_i_k_star[i, k+1]) * lambda_k[k]  for k in range(self.K_MOD - 1))
                            + φ_i_k_star[i, 0]
                            for i in range(self.num_agents))
                                  )
                  # Solve master problem
                  lambda_k.start = theta_solution[:self.K_MOD-1]
                  u_i.start = val_i
                  model.optimize()
                  theta_solution = np.array(model.x)

                  iter += 1


BundledChoiceKP.estimate_minmaxregret_BChoice = estimate_minmaxregret_BChoice

In [44]:
# example_pb = BundledChoiceKP(250, 30, [1, 2, 3, 4, 2.5], max_capacity= 10000)
# B_i_star = example_pb.linear_knapsack(torch.arange(example_pb.num_agents), example_pb.lambda_star)
# example_pb.generate_data_bundle_choice()
# example_pb.estimate_minmaxregret_BChoice(tol = 1e-4)

# characteristics:  5
φ_hat_k:  [ 493.02216 1005.65173 1481.5645  2005.933   1233.1968 ]
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2562382
Academic license 2562382 - for non-commercial use only - registered to ed___@nyu.edu
################
ITER: 0, parameters: [  82068.479 5877386.405 1623066.03   677831.925]
Value of LP:    -20.141682505607605
Reduced cost: 44868627.51104605
################
ITER: 1, parameters: [2041350.19  2457216.547 2481626.658 2083350.503]
Value of LP:    -187.44222847744823
Reduced cost: 8457128.194257714
################
ITER: 2, parameters: [2.061 3.051 4.077 2.591]
Value of LP:    -493.0217455987016
Reduced cost: 0.1129833245460361
################
ITER: 3, parameters: [2.004 3.007 4.01  2.506]
Value of LP:    -493.0217528691144
Reduced cost: 0.00012953767091516966
################
ITER: 4, parameters: [2.005 3.007 4.01  2.506]
Value of LP:    -493.0217528755531
Reduced cost: 5.125784511506026e-06
####################

## Matching

### Compute assignment large population

In [39]:
# def compute_assignment_large(self, max_iters = 100 ,tol = 1e-2 ):
#       with gp.Env(params=options) as env:
#           with gp.Model(env=env) as model:
#               ### Initialize
#               # Create variables
#               u_si = model.addVars(self.num_simulations * self.num_agents, lb = 0, ub = GRB.INFINITY , name="utilities")
#               p_j = model.addVars(self.num_objects, lb = 0, ub = GRB.INFINITY , name="prices")

#               # Set objective and initial constraints
#               model.setObjective( (1 / self.num_simulations) * u_si.sum() + p_j.sum(), GRB.MINIMIZE)
              
#               # Solve master problem 
#               model.setParam('OutputFlag', 0)
#               model.optimize()
#               theta_solution = np.array(model.x)

#               iter = 0
#               while iter < max_iters:
#                   print('################')
#                   ### Pricing problem
#                   p_j_iter = torch.tensor(theta_solution[-self.num_objects:], device= device , dtype=torch.float32)

#                   B_star_si = self.linear_knapsack(
#                                                   self.agents_si, 
#                                                   self.lambda_star, 
#                                                   eps_i_j = self.eps_si_j,
#                                                   p_j = p_j_iter
#                                                   )
                 
#                   φ_si_k_star = (self.φ_i_j_k[example_pb.agents_si] * B_star_si.unsqueeze(2)).sum(1)
               
#                   val_si =  φ_si_k_star @ self.lambda_star + (self.eps_si_j * B_star_si).sum(1) - (B_star_si * p_j_iter).sum(1)

#                   ### Stop if certificate holds
#                   certificate_i = val_si.cpu().numpy() - theta_solution[ : -self.num_objects]
#                   max_certificate = np.max(certificate_i)

#                   print("Value of LP:   ", model.objVal)
#                   print(f"Reduced cost: {max_certificate}")

#                   if max_certificate < tol:
#                     print('#############################################')
#                     print("SOLUTION FOUND")
#                     print('#############################################')
#                     return np.array(model.x), np.array(model.pi)
#                     break

#                   ### Master problem
#                   # Add constraints

#                   B_star_si = B_star_si.cpu().numpy()

#                   model.addConstrs(
#                             (u_si[si] + gp.quicksum(p_j[j] for j in np.where(B_star_si[si])[0])
#                              >= φ_si_k_star[si] @ self.lambda_star
#                             + self.eps_si_j[si, B_star_si[si]].sum()
#                             for si in range(self.num_simulations * self.num_agents))
#                               )

       
#                   # Solve master problem
                  
#                   u_si.start = val_si.cpu().numpy() 
#                   model.optimize()
#                   theta_solution = np.array(model.x)

#                   iter += 1


# BundledChoiceKP.compute_assignment_large = compute_assignment_large

In [40]:
def compute_assignment_large(self, max_iters = 100 ,tol = 1e-2 ):
      with gp.Env(params=options) as env:
          with gp.Model(env=env) as model:
              ### Initialize
              # Create variables
              u_si = model.addVars(self.num_simulations * self.num_agents, lb = 0, ub = GRB.INFINITY , name="utilities")
              p_j = model.addVars(self.num_objects, lb = 0, ub = GRB.INFINITY , name="prices")

              # Set objective and initial constraints
              model.setObjective( (1 / self.num_simulations) * u_si.sum() + p_j.sum(), GRB.MINIMIZE)
              
              # Solve master problem 
              model.setParam('OutputFlag', 0)
              model.optimize()
              solution_iter = np.array(model.x)

              iter = 0
              while iter < max_iters:
                  print('################')
                  ### Pricing problem
                  p_j_iter = torch.tensor(solution_iter[-self.num_objects:], device= device , dtype=torch.float32)

                  B_star_si, val_si = self.linear_knapsack(
                                                  self.agents_si, 
                                                  self.lambda_star, 
                                                  eps_i_j = self.eps_si_j,
                                                  p_j = p_j_iter,
                                                  return_value = True
                                                  )

                  ### Stop if certificate holds
                  certificate_i = val_si.cpu().numpy() - solution_iter[ : -self.num_objects]
                  max_certificate = np.max(certificate_i)

                  print(f"Value of LP:  {model.objVal}")
                  print(f"Reduced cost: {max_certificate}")

                  if max_certificate < tol:
                    print('#############################################')
                    print("SOLUTION FOUND")
                    print('#############################################')
                    return np.array(model.x), np.array(model.pi), iter +1

                  ### Master problem
                  # Add constraints
                  Φ_si_B_star = val_si + (B_star_si * p_j_iter).sum(1)
                  B_star_si = B_star_si.cpu().numpy()

                  model.addConstrs(
                                  (u_si[si] + gp.quicksum(p_j[j] for j in np.where(B_star_si[si])[0])
                                  >= Φ_si_B_star[si]
                                  for si in range(self.num_simulations * self.num_agents))
                                  )
                  
                  # Solve master problem
                  u_si.start = val_si.cpu().numpy() 
                  p_j.start = p_j_iter.cpu().numpy()
                  model.optimize()
                  solution_iter = np.array(model.x)
                  
                  iter += 1


BundledChoiceKP.compute_assignment_large = compute_assignment_large

In [41]:
# example_pb = BundledChoiceKP(10, 30, [1, 2, 3, 4, 2.5], num_simulations= 30, max_capacity= 200)
# primal_solution, dual_solution , iters = example_pb.compute_assignment_large()

### Estimate assignment

In [42]:
# def estimate_GMM_matching(self, max_iters = 100, tol = 1e-2 ):
#       with gp.Env(params=options) as env:
#           with gp.Model(env=env) as model:
#               ### Initialize
#               # Create variables
#               lambda_k = model.addVars(self.K_MOD , lb= 0, ub = 1e6 , name="parameters")
#               u_si = model.addVars(self.num_simulations * self.num_agents, lb = 0, ub = GRB.INFINITY , name="utilities")
#               p_j = model.addVars(self.num_objects, lb = 0, ub = GRB.INFINITY , name="prices")

#               # Set objective and initial constraints
#               model.setObjective(gp.quicksum( self.φ_hat_k[k] * lambda_k[k] for k in range(self.K_MOD ))
#                                   - (1 / self.num_simulations) * u_si.sum() - p_j.sum(), GRB.MAXIMIZE)

#               # model.addConstrs(
#               #                 (u_si[si] + gp.quicksum(p_j[j] for j in np.where(self.B_i_j_hat[si // self.num_simulations])[0])
#               #                 >= gp.quicksum((self.φ_hat_i_k[si // self.num_simulations, k]) * lambda_k[k]  for k in range(self.K_MOD))
#               #                 + self.eps_si_j[si, self.B_i_j_hat[si // self.num_simulations]].sum()
#               #                 for si in range(self.num_simulations * self.num_agents))
#               #                 )
              
#               # Solve master problem 
#               model.setParam('OutputFlag', 0)
#               model.optimize()
#               solution_iter = np.array(model.x)

#               iter = 0
#               while iter < max_iters:
#                   print('################')
#                   print(f"ITER: {iter}, parameters: {solution_iter[:self.K_MOD].round(3)}")

#                   ### Pricing problem
#                   lambda_k_iter = torch.tensor(solution_iter[:self.K_MOD], device= device , dtype=torch.float32)
#                   p_j_iter = torch.tensor(solution_iter[-self.num_objects:], device= device , dtype=torch.float32)

#                   B_star_si, val_si = self.linear_knapsack(
#                                                   self.agents_si, 
#                                                   lambda_k_iter, 
#                                                   eps_i_j = self.eps_si_j,
#                                                   p_j = p_j_iter,
#                                                   return_value = True
#                                                   )

#                   ### Stop if certificate holds
#                   certificate_i = val_si.cpu().numpy() - solution_iter[self.K_MOD: - self.num_objects]
#                   max_certificate = np.max(certificate_i)

#                   print("Value of LP:   ", model.objVal)
#                   print(f"Reduced cost: {max_certificate}")
#                   if max_certificate < tol:
#                     print('#############################################')
#                     print("Solution:       ", solution_iter[:self.K_MOD].round(3))
#                     print("True parameters:", self.lambda_star_np)
#                     print('#############################################')
#                     break

#                   ### Master problem
#                   # Add constraints
#                   φ_si_k_star = (self.φ_i_j_k[self.agents_si] * B_star_si.unsqueeze(2)).sum(1)
#                   B_star_si = B_star_si.cpu().numpy()

#                   model.addConstrs(
#                                   (u_si[si] + gp.quicksum(p_j[j] for j in np.where(B_star_si[si])[0])
#                                   >= gp.quicksum((φ_si_k_star[si, k]) * lambda_k[k]  for k in range(self.K_MOD))
#                                   + self.eps_si_j[si, B_star_si[si]].sum()
#                                   for si in range(self.num_simulations * self.num_agents))
#                                   )
                  
#                   # Solve master problem
#                   lambda_k.start = solution_iter[ : self.K_MOD]
#                   p_j.start = p_j_iter.cpu().numpy()
#                   u_si.start = val_si.cpu().numpy() 

#                   model.optimize()
#                   solution_iter = np.array(model.x)

#                   iter += 1

# BundledChoiceKP.estimate_GMM_matching = estimate_GMM_matching

In [43]:
def estimate_GMM_matching(self, max_iters = 100, tol = 1e-2 ):
      with gp.Env(params=options) as env:
          with gp.Model(env=env) as model:
              ### Initialize
              # Create variables
              lambda_k = model.addVars(self.K_MOD , lb= 0, ub = 1e6 , name="parameters")
              u_si = model.addVars(self.num_simulations * self.num_agents, lb = 0, ub = GRB.INFINITY , name="utilities")
              p_j = model.addVars(self.num_objects, lb = 0, ub = GRB.INFINITY , name="prices")

              # Set objective and initial constraints
              model.setObjective(gp.quicksum( self.φ_hat_k[k] * lambda_k[k] for k in range(self.K_MOD ))
                                  - (1 / self.num_simulations) * u_si.sum() - p_j.sum(), GRB.MAXIMIZE)

              # Solve master problem 
              model.setParam('OutputFlag', 0)
              model.optimize()
              solution_iter = np.array(model.x)

              iter = 0
              while iter < max_iters:
                  print('################')
                  print(f"ITER: {iter}, parameters: {solution_iter[:self.K_MOD]}")
          
                  ### Pricing problem
                  lambda_k_iter = torch.tensor(solution_iter[:self.K_MOD], device= device , dtype=torch.float32)
                  p_j_iter = torch.tensor(solution_iter[-self.num_objects:], device= device , dtype=torch.float32)

                  B_star_si, val_si = self.linear_knapsack(
                                                            self.agents_si, 
                                                            lambda_k_iter, 
                                                            eps_i_j = self.eps_si_j,
                                                            p_j = p_j_iter,
                                                            return_value = True
                                                            )

                  ### Stop if certificate holds
                  certificate_i = val_si.cpu().numpy() - solution_iter[self.K_MOD: - self.num_objects]
                  max_certificate = np.max(certificate_i)

                  print("Value of LP:   ", model.objVal)
                  print(f"Reduced cost: {max_certificate}")
                  if max_certificate < tol:
                        print('#############################################')
                        print("Solution:       ", solution_iter[:self.K_MOD].round(3))
                        print("True parameters:", self.lambda_star_np)
                        print('#############################################')
                        return solution_iter

                  ### Master problem
                  # Add constraints
                  φ_si_k_star = (self.φ_i_j_k[self.agents_si] * B_star_si.unsqueeze(2)).sum(1)
                  B_star_si = B_star_si.cpu().numpy()

                  model.addConstrs(
                                  (u_si[si] + gp.quicksum(p_j[j] for j in np.where(B_star_si[si])[0])
                                  >= gp.quicksum((φ_si_k_star[si, k]) * lambda_k[k]  for k in range(self.K_MOD))
                                  + self.eps_si_j[si, B_star_si[si]].sum()
                                  for si in range(self.num_simulations * self.num_agents))
                                  )
                  
                  # Solve master problem
                  lambda_k.start = solution_iter[ : self.K_MOD]
                  p_j.start = p_j_iter.cpu().numpy()
                  u_si.start = val_si.cpu().numpy() 

                  model.optimize()
                  solution_iter = np.array(model.x)

                  iter += 1

BundledChoiceKP.estimate_GMM_matching = estimate_GMM_matching